In [24]:
import imageio.v3 as iio
import matplotlib.pyplot as plt
import requests
import numpy as np
import os
from dotenv import load_dotenv
from datetime import datetime
import sys  
import colorsys
import csv
%matplotlib inline

load_dotenv()

True

In [25]:
# make api call to flickr
# for some reason, flickr key isn't found (KeyError)
FLICKR_API_KEY = os.getenv("FLICKR_API_KEY")

flickr_api_url = "https://www.flickr.com/services/rest/?method=flickr.photos.search" +\
            "&api_key=" + FLICKR_API_KEY +\
            "&tags=sunset" +\
            "&text=sunset%20landscape" +\
            "&content_type=1" +\
            "&has_geo=1" +\
            "&sort=date-posted-desc" +\
            "min_upload_date=2005-01-01" +\
            "&extras=date_taken%2C+geo%2C+url_o" +\
            "&format=json&nojsoncallback=1" +\
            "page=1"
            # "&is_getty=1" +\             

flickr_res = requests.get(flickr_api_url)
if(flickr_res.status_code < 200 or flickr_res.status_code >= 300): 
    print('Unsuccessful request. Error code: {}'.format(flickr_res.status_code))
    sys.exit()
else:
    print("Flickr API call was successful.")
    flickr_res = flickr_res.json()


f8ac40eee0aacfa3be6dee0cbac63930
Flickr API call was successful.


In [26]:
print(flickr_res["photos"]['photo'][0])

{'id': '52263583156', 'owner': '183902936@N04', 'secret': 'd315d5c9f6', 'server': '65535', 'farm': 66, 'title': 'Mindfulness', 'ispublic': 1, 'isfriend': 0, 'isfamily': 0, 'datetaken': '2022-08-01 19:07:51', 'datetakengranularity': 0, 'datetakenunknown': '0', 'latitude': '39.236554', 'longitude': '20.476155', 'accuracy': '14', 'context': 0, 'place_id': '', 'woeid': '265149', 'geo_is_public': 1, 'geo_is_contact': 0, 'geo_is_friend': 0, 'geo_is_family': 0, 'url_o': 'https://live.staticflickr.com/65535/52263583156_87dc5b2419_o.jpg', 'height_o': 1248, 'width_o': 1810}


In [68]:
# write csv header
fields = ['calc_score', 'humidity', 'cloud_cover', 'visibility', 'user_score', 'lat', 'long', 'date_time', 'image_url']

with open('./data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    row = ['a', 'b', 'c']
    writer.writerow(fields)

In [ ]:
# process multiple images at once
VISUALCROSSING_API_KEY = os.getenv("VISUALCROSSING_API_KEY")
DATE_TIME_FORMAT_STRING = "%Y-%m-%d %H:%M:%S"
SAMPLE_SCALE = 5

def score_sunset(image):
    sunset_pixel_counter = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rVal = image[i,j,0]
            gVal = image[i,j,1]  
            bVal = image[i,j,2]
            hVal = colorsys.rgb_to_hls(rVal, gVal, bVal)[0]*360
            if(hVal <= 45): # oranges are weighted less
                sunset_pixel_counter += 0.7
            elif(hVal >= 270): # reds and magentas are weighted fully
                sunset_pixel_counter +=1

    total_image_pixels = image.shape[1] *  image.shape[0]
    sunset_score = ((sunset_pixel_counter / total_image_pixels) * 10)
    sunset_score = round(sunset_score, 3)
    return sunset_score

f = open('./data.csv', 'a', newline='')
writer = csv.writer(f)

print("Length: ", len(flickr_res['photos']['photo']))

for i in range(len(flickr_res['photos']['photo'])):
    # Get and load pic from flickr res
    photo = flickr_res['photos']['photo'][i]
    imgUrl = f'https://live.staticflickr.com/{photo["server"]}/{photo["id"]}_{photo["secret"]}_b.jpg'
    pic = iio.imread(imgUrl)
    
    # GET WEATHER DATA FOR IMAGE
    # nearest hour of sunset for picture
    date_taken = datetime.strptime(photo["datetaken"], DATE_TIME_FORMAT_STRING)
    nearest_sunset_hour = date_taken.hour
    if date_taken.minute > 30:
        nearest_sunset_hour +=  1
    
    # make api call for picture
    visualcrossing_history_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?" +\
                            "&aggregateHours=1" +\
                            "&startDateTime=" + str(date_taken.date()) +\
                            "&endDateTime=" + str(date_taken.date())  +\
                            "&unitGroup=metric" +\
                            "&contentType=json" +\
                            "&dayStartTime=" + str(nearest_sunset_hour) + ":0:00" +\
                            "&dayEndTime=" + str(nearest_sunset_hour + 1) + ":0:00" +\
                            "&location=" + photo["latitude"] + "%2C%20" + photo["longitude"] +\
                            "&key=" + VISUALCROSSING_API_KEY
    weather_res = requests.get(visualcrossing_history_api_url)
    # print(type(weather_res))
    # print(weather_res.status_code)
    if(weather_res.status_code != 200): 
        print('Unsuccessful request. Error code: {}'.format(weather_res.status_code))
        sys.exit()
    else:
        # print('Visual Crossing API call was successful for index ', i)
        weather_res = weather_res.json()
    
    # get weather data of picture
    try:
        nearest_hour_data = weather_res['locations'][photo["latitude"] + ", " + photo["longitude"]]['values'][0]
        humidity = nearest_hour_data['humidity']
        cloud_cover = nearest_hour_data['cloudcover']
        visibility = nearest_hour_data['visibility']
    except KeyError:
        if(weather_res['errorCode'] == -994): # api call quota hit
            print(weather_res)
            break
        else:
            # handles errorCode 999 (weird stuff)
            print(weather_res)
        
        

    # prevent images with empty weather data from being written
    if(humidity == None or cloud_cover == None or visibility == None):
        print("image" , i, "doesn't have all data:")
        print(humidity, cloud_cover, visibility)
        continue

    # ANALYZE PHOTO
    length = pic.shape[0]
    width = pic.shape[1]
    sys_sample_pic = pic[0:length:SAMPLE_SCALE, 0:width:SAMPLE_SCALE, :]
    calc_score = score_sunset(sys_sample_pic)

    # WRITE TO CSV
    row = [calc_score, humidity, cloud_cover, visibility, "", photo["latitude"], photo["longitude"], date_taken, imgUrl]
    writer.writerow(row)
f.close()

In [ ]:
# make api call to visualcrossing for given photo
VISUALCROSSING_API_KEY = os.environ["VISUALCROSSING_API_KEY"]
aPhoto = flickr_res['photos']['photo'][3]
format_string = "%Y-%m-%d %H:%M:%S"
date_taken = datetime.strptime(aPhoto["datetaken"], format_string)
# print(type(aPhoto))
# print(aPhoto)
# print(aPhoto["latitude"])
# print(aPhoto["longitude"])
# print(aPhoto["datetaken"])
# print(str(date_taken.date()))
# print(str(date_taken.time()))

# nearest hour of sunset for picture taken
nearest_sunset_hour = date_taken.hour
if date_taken.minute > 30:
    nearest_sunset_hour +=  1
# print(nearest_sunset_hour)

visualcrossing_timeline_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" +\
                        aPhoto["latitude"] + "%2C%20" + aPhoto["longitude"] + "/" +\
                        str(date_taken.date()) + "/" + str(date_taken.date()) +\
                        "?unitGroup=us" +\
                        "&key=" + VISUALCROSSING_API_KEY +\
                        "&contentType=json"

visualcrossing_history_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?" +\
                            "&aggregateHours=1" +\
                            "&startDateTime=" + str(date_taken.date()) +\
                            "&endDateTime=" + str(date_taken.date())  +\
                            "&unitGroup=metric" +\
                            "&contentType=json" +\
                            "&dayStartTime=" + str(nearest_sunset_hour) + ":0:00" +\
                            "&dayEndTime=" + str(nearest_sunset_hour + 1) + ":0:00" +\
                            "&location=" + aPhoto["latitude"] + "%2C%20" + aPhoto["longitude"] +\
                            "&key=" + VISUALCROSSING_API_KEY

weather_res = requests.get(visualcrossing_history_api_url)

if(weather_res.status_code < 200 or weather_res.status_code >= 300): 
    print('Unsuccessful request. Error code: {}'.format(weather_res.status_code))
    sys.exit()
weather_res = weather_res.json()

In [ ]:
# write weather data of picture to database

# TODO: determine formatting for data
nearest_hour_data = weather_res["days"][0]["hours"][nearest_sunset_hour]
# print(nearest_hour_data)
print('Humidity : {}'.format(nearest_hour_data['humidity']))
print('Cloud Cover : {}'.format(nearest_hour_data['cloudcover']))
print('Sunset Time : {}'.format(weather_res['days'][0]['sunset']))
print(nearest_sunset_hour)


In [ ]:
# get systematic sample of pixels from image to analyze
aPhoto = flickr_res['photos']['photo'][1]
aImgUrl = f'https://live.staticflickr.com/{aPhoto["server"]}/{aPhoto["id"]}_{aPhoto["secret"]}_b.jpg'
aPic = iio.imread(aImgUrl)



# display photo
aFig = plt.figure(figsize=(10, 10))
fig = plt.figure()
plt.imshow(aPic)